# Quantum Circuit Builder

In this notebook, I try to make a quantum circuit builder

In [65]:
# Importing libraries
import numpy as np
import qutip as q
import matplotlib.pyplot as plt
from qutip import gates
#from qutip.qip.operations import *
#from qutip_qip.circuit import QubitCircuit

### Numpy

The first approach is to use numpy for everything (that is no QuTip)

In [66]:
# Defining qbit states 0 and 1
bra0 = np.array([[1, 0]])
ket0 = bra0.T
bra1 = np.array([[0, 1]])
ket1 = bra1.T
# Defining identity operator and Pauli operators
I = np.eye(2)
sigx = np.array([[0, 1], [1, 0]])
sigy = np.array([[0, -1j], [1j, 0]])
sigz = np.array([[1, 0], [0, -1]])
# Defining gate operators H, S, T, CNOT, and SWAP
H = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
S = np.array([[1, 0], [0, 1j]])
T = np.array([[1, 0], [0, np.exp(1j * np.pi / 4)]])
CNOT = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
SWAP = np.array([[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]])

After defining all the relevant gates, I define a function that applies single qubit gates on each qubit and then takes the kronecker product in the end

In [147]:
def QuantumCircuit(states, gates, targets):
    
    """
    Applies a sequence of gates to specified qubits in a quantum state.
    
    Parameters:
    states (numpy.ndarray): The initial quantum states.
    gates (list): A list of gate operators to apply.
    targets (list): A list of target qubit indices for each gate.
    
    Returns:
    numpy.ndarray: The final quantum state after applying all gates.
    """

    # Determine the number of qubits
    N = int(np.size(states)/2)

    # Check if the number of gates matches the number of targets
    if len(gates) != len(targets):
        raise ValueError("The number of gates must match the number of targets.")
    # Check if any of the targets are out of range
    for target in targets:
        if target < 0 or target >= N:
            raise ValueError(f"Target {target} is out of range for {N} qubits.")
    
    for i in range(len(gates)):
        temp = gates[i] @ states[targets[i]]
        states[targets[i]] = temp / np.linalg.norm(temp)  # Normalize the state

    return states

In [150]:
# Define states, gates, and targets
states = [ket0, ket0]  # Initial state |00>
gates = [H] # Apply Hadamard gates to both qubits
targets = [0]  # Apply to both qubits

In [151]:
QuantumCircuit(states, gates, targets)

[array([[0.70710678],
        [0.70710678]]),
 array([[1],
        [0]])]

Okay, this is all fine and well, but we need 2-qubit gates as well. To implement these, one has to use kroneker products for all the unitaries and states.

Now the function bellow is used to generate the unitary from single-qubit gates.

In [159]:
def generateUnitary(gates, targets, N):
    """
    Applies given single-qubit gates to specified qubit positions in an N-qubit system.

    Parameters:
        gates (list of np.ndarray): Single-qubit gate matrices.
        targets (list of int): Qubit indices (1-based). 0 means apply to all qubits.
        N (int): Total number of qubits.

    Returns:
        np.ndarray: Full operator as a Kronecker product.
    """
    if len(gates) != len(targets):
        raise ValueError("The number of gates and targets must be the same.")

    # Start with identity on all qubits
    operators = [I for _ in range(N)]

    for gate, target in zip(gates, targets):
        if target == 0:
            # Apply to all qubits
            for i in range(N):
                operators[i] = gate
        else:
            # Apply to specific (1-indexed) qubit
            if not (1 <= target <= N):
                raise ValueError(f"Target index {target} out of bounds for {N} qubits.")
            operators[target - 1] = gate

    # Compute Kronecker product (qubit 1 = leftmost)
    result = operators[0]
    for op in operators[1:]:
        result = np.kron(result, op)

    return result

In [154]:
# Define states, gates, and targets
states = [ket0, ket0] # Initial state |00>
gates = [H, H] # Apply Hadamard gates to both qubits
targets = [1, 2]  # Apply to both qubits

In [158]:
generateUnitary(gates, targets, 2)

array([[ 0.5,  0.5,  0.5,  0.5],
       [ 0.5, -0.5,  0.5, -0.5],
       [ 0.5,  0.5, -0.5, -0.5],
       [ 0.5, -0.5, -0.5,  0.5]])

In [133]:
def QuantumCircuit(states, gates, targets):
    
    """
    Applies a sequence of gates to specified qubits in a quantum state.
    
    Parameters:
    states (list): The initial qubit states.
    gates (list): A list of gate operators to apply.
    targets (list): A list of target qubit indices for each gate.
    
    Returns:
    numpy.ndarray: The final quantum state after applying all gates.
    """

    # Determine the number of qubits
    N = int(np.size(states)/2)

    state = states[0]
    for i in range(N-1):
        state = np.kron(state, states[i])

    # Check if the number of gates matches the number of targets
    if len(gates) != len(targets):
        raise ValueError("The number of gates must match the number of targets.")
    # Apply the gates to the specified qubits
    for i in range(len(gates)):
        gate = gates[i]
        gate = GateOnQubit(gate, N, targets[i])
        state = np.dot(gate, state)
        print(f"Quantum state after applying gate {i+1} to qubit {targets[i]}:")
        print(state)
    return state

In [137]:
QuantumCircuit(states, gates, targets)

Quantum state after applying gate 1 to qubit 1:
[[0.70710678+0.j]
 [0.70710678+0.j]
 [0.        +0.j]
 [0.        +0.j]]
Quantum state after applying gate 2 to qubit 2:
[[0.70710678+0.j]
 [0.70710678+0.j]
 [0.        +0.j]
 [0.        +0.j]]


array([[0.70710678+0.j],
       [0.70710678+0.j],
       [0.        +0.j],
       [0.        +0.j]])

In [33]:
GateOnQubit(sigx, 2, 0)

ValueError: n must be in the range [0, N-1]

In [ ]:
gates[0]

In [127]:
len(gates)

2

In [128]:
for i in range(len(gates)):
    temp = gates[i] @ states[targets[i]]
    states[targets[i]] = temp / np.linalg.norm(temp)  # Normalize the state

In [129]:
states

[array([[0.70710678],
        [0.70710678]]),
 array([[0.70710678],
        [0.70710678]])]

In [121]:
H @ states[0]

array([[0.70710678],
       [0.70710678]])

In [120]:
states

[array([[1],
        [0]]),
 array([[1],
        [0]])]

In [123]:
QuantumCircuit(state, gates, targets)

After applying gate 1 to qubit 0:
[[0.70710678+0.j]
 [0.70710678+0.j]
 [0.        +0.j]
 [0.        +0.j]]
After applying gate 2 to qubit 1:
[[0.70710678+0.j]
 [0.70710678+0.j]
 [0.        +0.j]
 [0.        +0.j]]


array([[0.70710678+0.j],
       [0.70710678+0.j],
       [0.        +0.j],
       [0.        +0.j]])

In [114]:
def QuantumCircuit(state, gates, targets):
    
    """
    Applies a sequence of gates to specified qubits in a quantum state.
    
    Parameters:
    state (numpy.ndarray): The initial quantum state vector.
    gates (list): A list of gate operators to apply.
    targets (list): A list of target qubit indices for each gate.
    
    Returns:
    numpy.ndarray: The final quantum state after applying all gates.
    """

    # Determine the number of qubits
    N = int(np.size(state)/2)

    # Check if the number of gates matches the number of targets
    if len(gates) != len(targets):
        raise ValueError("The number of gates must match the number of targets.")
    # Check if any of the targets are out of range
    for target in targets:
        if target < 0 or target >= N:
            raise ValueError(f"Target {target} is out of range for {N} qubits.")

    for i in range(len(gates)):
        gate = gates[i]
        gate = GateOnQubit(gate, N, targets[i])
        state = np.dot(gate, state)
        print(f"After applying gate {i+1} to qubit {targets[i]}:")
        print(state)
    return state

In [148]:
sigz

array([[ 1,  0],
       [ 0, -1]])

In [149]:
apply_gate_to_qubit(sigx, 4, 2)  # Applying Pauli-X gate to the first qubit in a 2-qubit system

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

In [85]:
I_N[0:3, 0:3]

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [ ]:
N = 2  # Number of qubits
n = 0  # Index of the qubit to apply the operator to

np.array([I, sigx, I, I])



np.kron(a, b, c, d)



ValueError: could not broadcast input array from shape (2,2) into shape (1,1)

np.eye(2**4)

In [70]:
def SigmaX(N, n):
    """
    Returns the Pauli-X operator for qubit n in an N-qubit system.
    """
    if n < 0 or n >= N:
        raise ValueError("n must be in the range [0, N-1]")
    op = np.eye(2*N)
    op[2**n:2**(n+1), 2**n:2**(n+1)] = sigx
    return op

In [57]:
N

1

In [58]:
gates = np.array([sigx, H])
states = np.array([ket0, ket0])

# Determine the number of qubits
N = len(states)  # Example for two qubits


# Defining a function to apply an array of gates to an array of input states
def apply_gates(gates, states):
    result = []
    for state in states:
        for gate in gates:
            state = np.dot(gate, state)
        result.append(state)
    return np.array(result)
# Applying gates to states
result = apply_gates(gates, states)
# Printing the result
print("Resulting states after applying gates:")
for i, state in enumerate(result):
    print(f"State {i+1}: {state}")




Resulting states after applying gates:
State 1: [ 0.70710678 -0.70710678]
State 2: [ 0.70710678 -0.70710678]


In [49]:
# Defining qbit states 0 and 1
ket0 = q.basis(2, 0)
bra0 = ket0.dag()
ket1 = q.basis(2, 1)
bra1 = ket1.dag()
# Defining identity operator and Pauli operators
I = q.identity(2)
sigx = q.sigmax()
sigy = q.sigmay()
sigz = q.sigmaz()
# Defining gate operators
H = gates.hadamard_transform()
T = gates.t_gate()
S = gates.s_gate()
CNOT = gates.cnot()
SWAP = gates.swap()

In [52]:
CNOT

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=CSR, isherm=True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]